In [2]:
import numpy as np 
import pandas as pd 

In [3]:
# Collecting Data
train=pd.read_csv("../input/spaceship-titanic/train.csv")
test=pd.read_csv("../input/spaceship-titanic/test.csv")
train.info()

In [4]:
#Retreiving 
Y=train["Transported"]

In [5]:
train_1=train.drop(["Transported"],axis=1)
data=pd.concat([train_1,test],axis=0).reset_index(drop=True)
data.shape

In [6]:
data.isna().sum()

In [7]:
data_1 = data.copy()

# Listing Missing Values In Categorical Data
list_missing_cat_columns = list((data_1.select_dtypes(['object', 'category']).isna().sum() > 0).index)
list_missing_cat_columns

In [8]:
# Filling Missing Values in Categorical Data
for col in list_missing_cat_columns:
    data_1[col] = data_1[col].fillna(data_1[col].mode()[0])

In [9]:
# Checking Null Values
data_1.isna().sum()

In [10]:
# Listing Missing Values In Continuous Data
list_missing_numeric_col = list((data_1.select_dtypes(np.number).isna().sum() > 0).index)
list_missing_numeric_col

In [11]:
# Filling Missing Values Of Age Column
def fill_nans_by_age_and_cryosleep(df):
    df["RoomService"] = np.where((df["Age"] < 13) | (df["CryoSleep"] == True), 0, df["RoomService"])
    df["FoodCourt"] = np.where((df["Age"] < 13) | (df["CryoSleep"] == True), 0, df["FoodCourt"])
    df["ShoppingMall"] = np.where((df["Age"] < 13) | (df["CryoSleep"] == True), 0, df["ShoppingMall"])
    df["Spa"] = np.where((df["Age"] < 13) | (df["CryoSleep"] == True), 0, df["Spa"])
    df["VRDeck"] = np.where((df["Age"] < 13) | (df["CryoSleep"] == True), 0, df["VRDeck"])
    
    return df

data_1 = fill_nans_by_age_and_cryosleep(data_1)

In [12]:
# Filling Missing Values in Continuous Data
for col in list_missing_numeric_col:
    data_1[col] = data_1[col].fillna(data_1[col].mean())
data_1.isna().sum()

In [13]:
# Treating Outliers
def clipping_quantile(dataframe, quantile_values = None, quantile = 0.99):
    df = dataframe.copy()
    if quantile_values is None:
        quantile_values = df[["RoomService", "FoodCourt", "ShoppingMall", "Spa", "VRDeck"]].quantile(quantile)
    for num_column in ["RoomService", "FoodCourt", "ShoppingMall", "Spa", "VRDeck"]:
        num_values = df[num_column].values
        threshold = quantile_values[num_column]
        num_values = np.where(num_values > threshold, threshold, num_values)
        df[num_column] = num_values
    return df      
    
data_1 = clipping_quantile(data_1, None, 0.99)

In [15]:
data_2=data_1.copy()
data_2.info()

In [16]:
# Drop Name, PassengerId Columns
data_2=data_2.drop(['Name',"PassengerId",],axis=1)
data_2.info()

In [ ]:
# Drop Cabin Column
data_3=data_3.drop(["Cabin"],axis=1)
data_3.info()

In [17]:
# Getting Dummies
data_3 = pd.get_dummies(data_2, columns = ['HomePlanet', 'CryoSleep', 'Destination', 'VIP'])

In [19]:
train_data_final = data_3.loc[:train.index.max(),  :].copy()
test_data_final = data_3.loc[train.index.max() + 1:, :].reset_index(drop = True).copy()

In [20]:
# Final Data To Train Models
test_data_final.shape